In [1]:
import datetime
from dateutil.relativedelta import relativedelta

from marcap import marcap_data
import pandas as pd

# Why the hangul is not typed in vscode. 
# It's suck. 


# get date from macapdata
def get_data(start_day, end_day=None):
  if end_day == None:
    end_day = datetime.date.today() - relativedelta(days=1)
  
  # get while data form marcap.
  whole_from = start_day - relativedelta(years=1) - relativedelta(days=1)
  whole_to = end_day 
  whole_df = marcap_data(whole_from, whole_to)
  
  result_df = None
  day = start_day
  result_df = pd.DataFrame()
  while day <= end_day:
    try:    
      day_df = whole_df.loc[str(day)]
    except KeyError:
      day = day + relativedelta(days=1)
      continue


    day_df = day_df.set_index("Code")
    day_df.insert(0,'Date',day)

    year_from= day- relativedelta(years=1) - relativedelta(days=1)
    year_to = day- relativedelta(days=1)
    # print("getting data for ", day)
    # print("dataset is from ", year_from, "to", year_to)

    year_df = whole_df.loc[year_from : year_to]
    year_df = year_df[['Code', 'Name', 'Close']]
    gg = year_df.groupby(['Code'])
    first_df = gg.first().rename(columns={'Close':'1-YrFirst'})
    max_df = gg.max().rename(columns={'Close':'1-YrHigh'})
    min_df = gg.min().rename(columns={'Close':'1-YrLow'})
    # count_df = gg.count().rename(columns={'Close':'Count'})
  

    # merge aggregated values min/max/first
    day_df = pd.merge(day_df, first_df)
    day_df = pd.merge(day_df, min_df)
    day_df = pd.merge(day_df, max_df)
    # day_df = pd.merge(day_df, count_df)
  
    day_df['Result'] = False
    for index, co in day_df.iterrows():
      if co['1-YrFirst'] >= co['1-YrHigh']:
        if co['1-YrFirst'] < co['Close']:
          day_df.at[index,'Result'] = True
  
    # filtering
    mask = day_df.Result == True
    day_df = day_df.loc[mask,:]

    result_df =  pd.concat([result_df, day_df], join="outer")
    day = day + relativedelta(days=1)
  return result_df 


In [2]:
today = datetime.date.today()
start_day = today - relativedelta(days=30)
end_day = today - relativedelta(days=1)
df = get_data(start_day)

In [3]:
df


,Date,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,...,Volume,Amount,Marcap,Stocks,MarketId,Rank,1-YrFirst,1-YrLow,1-YrHigh,Result
1131,2021-01-18,사조동아원,KOSPI,NaN,1055,1,15,1.44,1040,1105,...,4576469,4872307340,148907553000,141144600,STK,1168,1045,481,1045,True
2054,2021-01-18,성안,KOSPI,NaN,733,1,89,13.82,756,837,...,76984130,61423205326,41676811380,56857860,STK,2172,675,300,675,True
2112,2021-01-18,태원물산,KOSPI,NaN,4480,1,200,4.67,4240,4535,...,167599,742693300,34048000000,7600000,STK,2238,4460,2455,4460,True
307,2021-01-19,필룩스,KOSPI,NaN,8190,4,1890,30.00,6910,8190,...,140511066,1068638277950,737453218320,90043128,STK,317,7040,2925,7040,True
1946,2021-01-19,SJM홀딩스,KOSPI,NaN,3490,1,260,8.05,3260,3670,...,346997,1205657775,52119687920,14934008,STK,2053,3250,2005,3250,True
28,2021-01-20,아모레퍼시픽,KOSPI,NaN,229500,1,6500,2.91,222000,232500,...,425304,96768709000,13416223455000,58458490,STK,29,225000,147500,225000,True
1073,2021-01-20,메카로,KOSDAQ,우량기업부,16550,1,250,1.53,16500,16600,...,319912,5174350900,167192072000,10102240,KSQ,1102,16450,7290,16450,True
1136,2021-01-20,디와이,KOSPI,NaN,5800,1,770,15.31,5500,6530,...,9385506,57051751800,152653871400,26319633,STK,1173,5610,2310,5610,True
650,2021-01-21,연우,KOSDAQ,우량기업부,25600,1,2000,8.47,24600,25650,...,339286,8546030400,317388800000,12398000,KSQ,665,25050,14250,25050,True
1343,2021-01-21,비비씨,KOSDAQ,중견기업부,22400,1,150,0.67,22150,22750,...,228081,5054559100,121042499200,5403683,KSQ,1393,22300,18100,22300,True
